<a href="https://colab.research.google.com/github/ImagingDataCommons/ETL/blob/gw-sprint29/notebooks/clin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Get the scrapped clinical files, clinical note file

In [1]:
!git clone --branch gw https://github.com/fedorov/actcianable

Cloning into 'actcianable'...
remote: Enumerating objects: 3396, done.
remote: Counting objects: 100% (2527/2527), done.
remote: Compressing objects: 100% (1443/1443), done.
remote: Total 3396 (delta 1173), reused 2315 (delta 1040), pack-reused 869
Receiving objects: 100% (3396/3396), 176.59 MiB | 24.69 MiB/s, done.
Resolving deltas: 100% (1623/1623), done.


Get the ETL code to format the clinical files into json files, and create the BigQuery tables

In [2]:
!git clone --branch gw-sprint29 https://github.com/ImagingDataCommons/ETL.git

Cloning into 'ETL'...
remote: Enumerating objects: 461, done.
remote: Counting objects: 100% (461/461), done.
remote: Compressing objects: 100% (266/266), done.
remote: Total 461 (delta 249), reused 377 (delta 176), pack-reused 0
Receiving objects: 100% (461/461), 3.41 MiB | 20.08 MiB/s, done.
Resolving deltas: 100% (249/249), done.


Get required packages. Restart runtime right after. No need to rerun cells above 

In [3]:
%cd ETL/clinical
!pip3 install -r requirements.txt

/content/ETL/clinical
     |████████████████████████████████| 15.7 MB 5.6 MB/s 
     |████████████████████████████████| 503 kB 56.1 MB/s 
     |████████████████████████████████| 96 kB 6.0 MB/s 
  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.15.0
  Attempting uninstall: pytz
    Found existing installation: pytz 2018.9
    Uninstalling pytz-2018.9:
      Successfully uninstalled pytz-2018.9
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.3.post1 requires numpy<1.20,>=1.

Parse the scrapped the Excel files loaded from the github into Json files 

In [1]:
%cd ETL/clinical/
!mkdir clin
!python3 parse_clinical_files.py /content/actcianable/output/

/content/ETL/clinical
strcInfo {'filenm': 'TCIA Breast clinical data public 7_16_11.xlsx', 'sheet': 0, 'headrows': [0, 1], 'pivot': True}
/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
strcInfo {'filenm': 'SharedClinicalAndRFS.xls', 'sheet': 1, 'patientIdRow': 1, 'maxRow': 64, 'skipCols': [37, 38]}
strcInfo {'filenm': 'C4KC KiTS_Clinical Data_Version 1.csv', 'sheet': 0}
strcInfo {'filenm': 'COVID-19 AR Clinical Correlates July202020.xlsx', 'sheet': 0, 'headrows': [1]}
strcInfo {'filenm': 'COVID-19 AR Clinical Correlates July202020.xlsx', 'sheet': 1, 'headrows': [1]}
strcInfo {'filenm': 'deidentified_overlap_tcia.csv.cleaned.csv_20210806.csv', 'sheet': 0}
strcInfo {'filenm': 'clinicalapi-cptac.esacinc.com_api_tcia_clinical_data_tumor_code_AML', 'type': 'json'}
strcInfo {'filenm': 'clinicalapi-cptac.esacinc.com_api_tcia_clinical_data_tumor_code_CM', 'type': 'json'}
strcInfo {'filenm

Authenticate with an account that has access to the desired Google Cloud project to host the BigQuery Tables

In [2]:
from google.colab import auth
auth.authenticate_user()

Create a new dataset in BigQuery and populuate it with the clinical data in the Json files created above. For now a separate table is created for each collection. The table clinical_meta summarizes the data stored across all collections. The two arguments in the script below are the Google Cloud project and the dataset. If the dataset already exists then it is first deleted. 

In [3]:
!python3 bq_export.py idc-dev-etl clinical2